# 前処理
音声データを前処理する。
訓練データはtrain_details.csvを元に音声ファイルを分割する。
テストデータはinaSpeechSegmenterライブラリを使用して発話区間検出を行い、音声ファイルを分割する。

## ライブラリのimport

In [1]:
import glob
import os
import time
from concurrent.futures import ProcessPoolExecutor

import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

In [2]:
INPUT_PATH = "./data"

## 訓練データ

In [3]:
# 音声データのパスを取得
mp3_files = glob.glob(f"train/**/*.mp3", recursive=True)
print(mp3_files[0:3])
print(len(mp3_files))

['train/MnRSOJb1XP/MJq8kcDcLN/XxkgOdbThF/y68lVXZPkzA2tZVyhK4x.mp3', 'train/MnRSOJb1XP/KxHOxqQyjL/XxkgOdbThF/3JGeBMvPq1JOA3OyE2jj.mp3', 'train/6WkcCe8ba3/RmNQjCMhUi/pdI4lkOMy1/WjrIPWr6EvA5JrSpM7LD.mp3']
800


In [4]:
train = pd.read_csv(f"{INPUT_PATH}/train.csv")
train[0:3]

,ID,author,title,daisy_folder,audio_path,target
0,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...
1,06mgGNnTThgyIG6,w4mwiQxQI4,AYMK6FMxCG,9su1mGrvoq,train/w4mwiQxQI4/AYMK6FMxCG/9su1mGrvoq/pX3ipSW...,と自分は文政年間にできた地図で見たことがある小手指原久米川は古戦場なり源平小手指原にて戦うこ...
2,0MOLuKeb4EJvC45,PkyxkVV9Gq,cP82a8LULP,GZIf1u5UJu,train/PkyxkVV9Gq/cP82a8LULP/GZIf1u5UJu/ZXLCjee...,一あれ程迄世間を騒がせた天一坊も、とうとうお処刑となって、獄門に梟けられてしまいました。あの...


In [5]:
#訓練データの音声区切り目を付与
train_details = pd.read_csv(f"{INPUT_PATH}/train_details.csv")
train_details[0:3]

,DETAIL_ID,ID,slice_id,duration,start_time,end_time,target_slice
0,022CmvRb8aM5pKF_1,022CmvRb8aM5pKF,1,2533.878,0.000,2533.878,幸ありて
1,022CmvRb8aM5pKF_2,022CmvRb8aM5pKF,2,3631.020,2533.878,6164.898,普通の人ならたいして問題にすまいこのことが、
2,022CmvRb8aM5pKF_3,022CmvRb8aM5pKF,3,3082.449,6164.898,9247.347,私の心を暗くした。


In [6]:
# 結合
df_train = pd.merge(train,train_details,on="ID")
df_train[0:2]

,ID,author,title,daisy_folder,audio_path,target,DETAIL_ID,slice_id,duration,start_time,end_time,target_slice
0,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...,022CmvRb8aM5pKF_1,1,2533.878,0.000,2533.878,幸ありて
1,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...,022CmvRb8aM5pKF_2,2,3631.020,2533.878,6164.898,普通の人ならたいして問題にすまいこのことが、


In [7]:
#スライスした音声データの保存
!mkdir fine_tune
!ls

mkdir: ディレクトリ `fine_tune' を作成できません: ファイルが存在します
chutorial.ipynb  preprocess.ipynb   test_slice		vad.ipynb
data		 pyproject.toml     test_vad		whisper_finetune
fine_tune	 test		    train
poetry.lock	 test_rm_noise_vad  train_remove_noise


In [8]:
# 音声を細切れ(slice)にして、別のディレクトリへ保存
def slice_audio(file_path, start_time, end_time, output_dir, output_filename):
    # Load the audio file
    audio = AudioSegment.from_mp3(file_path)

    # Slice the audio file (convert times to milliseconds)
    sliced_audio = audio[start_time:end_time]

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file path
    output_path = os.path.join(output_dir, output_filename)

    # Export the sliced audio
    sliced_audio.export(output_path, format="mp3")

# 処理の実行
def process_audio_files(file_info):
    path, start_time, end_time, output_dir, output_filename = file_info
    slice_audio(path, start_time, end_time, output_dir, output_filename)

# 音声ファイルの情報を取得
list_audio_path = df_train["audio_path"].tolist()
file_info_list = [
    (
        path,
        df_train.iloc[i, df_train.columns.get_loc("start_time")],
        df_train.iloc[i, df_train.columns.get_loc("end_time")],
        "fine_tune/",
        df_train.iloc[i, df_train.columns.get_loc("DETAIL_ID")] + ".mp3"
    )
    for i, path in enumerate(list_audio_path)
]

start = time.time()

# 並列処理の設定（ProcessPoolExecutorを使用）
with ProcessPoolExecutor() as executor:
    list(tqdm(executor.map(process_audio_files, file_info_list), total=len(file_info_list)))

end = time.time()
print(f"処理時間: {end - start:.2f}秒")

100%|██████████| 354043/354043 [14:57:55<00:00,  6.57it/s]   


処理時間: 53891.64秒


## テストデータ

In [9]:
import os

from inaSpeechSegmenter import Segmenter
from tqdm import tqdm
from pydub import AudioSegment

2024-08-17 07:48:59.335580: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-17 07:48:59.536280: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-17 07:49:00.100472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
!mkdir test_vad
!ls

mkdir: ディレクトリ `test_vad' を作成できません: ファイルが存在します
chutorial.ipynb			  preprocess.ipynb   test_vad
data				  pyproject.toml     train
fine_tune			  test		     train_remove_noise
inference_baseline_whisper.ipynb  test_rm_noise_vad  vad.ipynb
poetry.lock			  test_slice	     whisper_finetune


In [11]:
df_test = pd.read_csv(f"{INPUT_PATH}/test.csv")
df_test[0:3]

,ID,author,title,daisy_folder,audio_path
0,06elw1oovcrMbjV,fdoFHXEq2E,ZW5E4286Nc,6Q3jYiU75i,test/fdoFHXEq2E/ZW5E4286Nc/6Q3jYiU75i/EQkWbbdY...
1,0PSPmAu5rkuqd30,CKVW6cIgxI,Q1etdEiaGT,08ei9SVc7t,test/CKVW6cIgxI/Q1etdEiaGT/08ei9SVc7t/3pNG5d8E...
2,0PmjHGRU8Md3RHO,GZADw2tadP,PpyqSKd1uC,hmh8it6Xmc,test/GZADw2tadP/PpyqSKd1uC/hmh8it6Xmc/MZ9owpTk...


In [12]:
for audio_path in tqdm(df_test["audio_path"].to_list(), total=len(df_test["audio_path"].to_list())):
    seg = Segmenter(vad_engine='smn', detect_gender=False)
    segmentation = seg(audio_path)

    audio = AudioSegment.from_mp3(audio_path)
    audio_file_name = os.path.basename(audio_path)
    count = 0

    # 各speechセグメントを抽出して保存
    for i, (seg_type, start, end) in enumerate(segmentation):
        if seg_type == 'speech':
            # セグメントの開始・終了時間をミリ秒に変換
            start_ms = int(start * 1000)
            end_ms = int(end * 1000)

            # 対応する音声部分を切り出す
            speech_segment = audio[start_ms:end_ms]

            # 出力ファイル名を生成
            output_file = f"test_vad/{audio_file_name}_{count+1}.mp3"

            # 切り出したセグメントを保存
            speech_segment.export(output_file, format="mp3")
            count += 1
            #print(f"Saved {output_file}")

  0%|          | 0/200 [00:00<?, ?it/s]/home/toshi/work/nishika/bungaku_asr/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2024-08-17 07:49:17.701617: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-17 07:49:17.722579: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to do

750/750 - 2s - 3ms/step


  0%|          | 1/200 [00:13<43:36, 13.15s/it]/home/toshi/work/nishika/bungaku_asr/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1093/1093 - 3s - 3ms/step


  1%|          | 2/200 [00:26<42:52, 12.99s/it]/home/toshi/work/nishika/bungaku_asr/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

 99%|█████████▉| 198/200 [1:10:20<00:44, 22.17s/it]/home/toshi/work/nishika/bungaku_asr/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63/63 - 0s - 6ms/step


100%|█████████▉| 199/200 [1:10:21<00:15, 15.99s/it]/home/toshi/work/nishika/bungaku_asr/.venv/lib/python3.11/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2034/2034 - 6s - 3ms/step


100%|██████████| 200/200 [1:10:52<00:00, 21.26s/it]


## 分割したデータの確認

In [1]:
import os
import glob
import pandas as pd

INPUT_PATH = "./data"

In [2]:
df_test = pd.read_csv(f"{INPUT_PATH}/test.csv")

In [3]:
for audio_path, id_name in zip(df_test["audio_path"].to_list(), df_test["ID"].to_list()):
    audio_file_name = os.path.basename(audio_path)
    file_pattern = os.path.join("test_vad", f"{audio_file_name}_*.mp3")
    audio_file_names = glob.glob(file_pattern)
    
    print(f"{audio_path} {id_name}: {len(audio_file_names)}")

test/fdoFHXEq2E/ZW5E4286Nc/6Q3jYiU75i/EQkWbbdYr7To54jFH8Mf.mp3 06elw1oovcrMbjV: 206
test/CKVW6cIgxI/Q1etdEiaGT/08ei9SVc7t/3pNG5d8E3LsyTtyrwlpQ.mp3 0PSPmAu5rkuqd30: 140
test/GZADw2tadP/PpyqSKd1uC/hmh8it6Xmc/MZ9owpTksjJXHfS6HiIn.mp3 0PmjHGRU8Md3RHO: 538
test/NlBaOYwpgD/k3X9ZDDpyW/cwiduwQhyG/vFovD5liGixQDKJjAEwX.mp3 0ehng36v9OQHEgF: 430
test/c0Fr0A61D3/OFaCYxVivl/OAIgZj971N/i9OQFwHZBYwR9lS2gd2E.mp3 0pRlQcSLrxRVdwc: 59
test/pp0PzIWycC/ECtb5xyliT/85zddEWVEZ/i8Q3GUqEY4YESN0z78T9.mp3 0yZpH8rpH6K50SG: 299
test/bl3EvKheJN/8ZPdcXr5j9/g6WKP2DFPh/VQAiBiY0SF3jE8UmFJph.mp3 13VFpPvfhlTi2jO: 77
test/SXTvUsglT5/M4pR9zSHPA/Uab1nTe4X1/Na6gR2CtwxNpKQViUNQs.mp3 1LySIBy31q1mQ1b: 114
test/SXTvUsglT5/EVvSbvDMur/d5Quo9IlsI/TXGtTY4vtkKG8OEUJSWp.mp3 1fbNiCS6CNexkGp: 250
test/jc8mfAHTeq/c46xQSa2lF/02aHhvnF2d/jMwBUkeuo4Cq31mn4eUB.mp3 1gIbc1hRD665U5W: 548
test/CKVW6cIgxI/SKmnXqK9go/FLoSHSeHqP/HQWaCQnNvMv0t1BNmPoW.mp3 1p23gsb9WO94Bse: 1
test/K4EnkdRBeE/y1eVWk8N29/Uwb5RBHYZN/UiDJKYX8CqedbutySMWS.mp3 1vmk9ImBm9o3JfB: 

分割に失敗しているファイルがあるので確認

## ノイズ除去してVADを行う

In [1]:
import subprocess
import os
import pandas as pd

from inaSpeechSegmenter import Segmenter
from pydub import AudioSegment
from tqdm import tqdm

INPUT_PATH = "./data"

2024-08-18 23:22:56.306869: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-18 23:22:56.375930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 23:22:56.453715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 23:22:56.481845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 23:22:56.549823: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def remove_noise(input_file, output_file):
    # ディレクトリ部分を取得
    directory = os.path.dirname(output_file)

    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)

    # FFmpegコマンドを作成
    command = [
        'ffmpeg',
        '-i', input_file,             # 入力ファイル
        '-af', 'afftdn=nf=-25', # "agate"    # ノイズ除去フィルタ (afftdn)
        output_file                   # 出力ファイル
    ]

    # FFmpegコマンドを実行
    subprocess.run(command, check=True)

In [4]:
!mkdir test_rm_noise
!mkdir test_vad_rm_noise
!ls

mkdir: ディレクトリ `test_vad_rm_noise' を作成できません: ファイルが存在します
chutorial.ipynb			  poetry.lock	     train
data				  preprocess.ipynb   train_llm_asr.py
fine_tune			  pyproject.toml     train_remove_noise
finetune_whisper.ipynb		  test		     train_whisper.py
inference_baseline_whisper.ipynb  test_rm_noise      vad.ipynb
inference_whisper.py		  test_slice	     vad_test.py
llm_asr				  test_vad	     whisper_finetune
output				  test_vad_rm_noise  whisper_finetune_exp001


In [3]:
df_test = pd.read_csv(f"{INPUT_PATH}/test.csv")
df_test[0:3]

,ID,author,title,daisy_folder,audio_path
0,06elw1oovcrMbjV,fdoFHXEq2E,ZW5E4286Nc,6Q3jYiU75i,test/fdoFHXEq2E/ZW5E4286Nc/6Q3jYiU75i/EQkWbbdY...
1,0PSPmAu5rkuqd30,CKVW6cIgxI,Q1etdEiaGT,08ei9SVc7t,test/CKVW6cIgxI/Q1etdEiaGT/08ei9SVc7t/3pNG5d8E...
2,0PmjHGRU8Md3RHO,GZADw2tadP,PpyqSKd1uC,hmh8it6Xmc,test/GZADw2tadP/PpyqSKd1uC/hmh8it6Xmc/MZ9owpTk...


In [5]:
for audio_path in tqdm(df_test["audio_path"].to_list(), total=len(df_test["audio_path"].to_list())):
    # ノイズ除去
    rm_noise_audio_path = audio_path.replace("test", "test_rm_noise")
    remove_noise(audio_path, rm_noise_audio_path)

    seg = Segmenter(vad_engine='smn', detect_gender=False)
    segmentation = seg(rm_noise_audio_path)

    audio = AudioSegment.from_mp3(rm_noise_audio_path)
    audio_file_name = os.path.basename(rm_noise_audio_path)
    count = 0

    # 各speechセグメントを抽出して保存
    for i, (seg_type, start, end) in enumerate(segmentation):
        if seg_type == 'speech':
            # セグメントの開始・終了時間をミリ秒に変換
            start_ms = int(start * 1000)
            end_ms = int(end * 1000)

            # 対応する音声部分を切り出す
            speech_segment = audio[start_ms:end_ms]

            # 出力ファイル名を生成
            output_file = f"test_vad_rm_noise/{audio_file_name}_{count+1}.mp3"

            # 切り出したセグメントを保存
            speech_segment.export(output_file, format="mp3")
            count += 1
            #print(f"Saved {output_file}")

  0%|          | 0/200 [00:00<?, ?it/s]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --

750/750 - 19s - 25ms/step


  0%|          | 1/200 [01:22<4:33:13, 82.38s/it]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-lib

1028/1028 - 11s - 11ms/step


  1%|          | 2/200 [02:31<4:05:49, 74.49s/it]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-lib

...

 99%|█████████▉| 198/200 [1:56:50<00:57, 28.95s/it]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-l

63/63 - 1s - 11ms/step


100%|█████████▉| 199/200 [1:56:52<00:20, 20.93s/it]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-l

2062/2062 - 2s - 807us/step


100%|██████████| 200/200 [1:57:24<00:00, 35.22s/it]


In [7]:
import glob

for audio_path, id_name in zip(df_test["audio_path"].to_list(), df_test["ID"].to_list()):
    audio_file_name = os.path.basename(audio_path)
    file_pattern = os.path.join("test_vad_rm_noise", f"{audio_file_name}_*.mp3")
    audio_file_names = glob.glob(file_pattern)
    
    print(f"{audio_path} {id_name}: {len(audio_file_names)}")

test/fdoFHXEq2E/ZW5E4286Nc/6Q3jYiU75i/EQkWbbdYr7To54jFH8Mf.mp3 06elw1oovcrMbjV: 205
test/CKVW6cIgxI/Q1etdEiaGT/08ei9SVc7t/3pNG5d8E3LsyTtyrwlpQ.mp3 0PSPmAu5rkuqd30: 173
test/GZADw2tadP/PpyqSKd1uC/hmh8it6Xmc/MZ9owpTksjJXHfS6HiIn.mp3 0PmjHGRU8Md3RHO: 592
test/NlBaOYwpgD/k3X9ZDDpyW/cwiduwQhyG/vFovD5liGixQDKJjAEwX.mp3 0ehng36v9OQHEgF: 456
test/c0Fr0A61D3/OFaCYxVivl/OAIgZj971N/i9OQFwHZBYwR9lS2gd2E.mp3 0pRlQcSLrxRVdwc: 75
test/pp0PzIWycC/ECtb5xyliT/85zddEWVEZ/i8Q3GUqEY4YESN0z78T9.mp3 0yZpH8rpH6K50SG: 345
test/bl3EvKheJN/8ZPdcXr5j9/g6WKP2DFPh/VQAiBiY0SF3jE8UmFJph.mp3 13VFpPvfhlTi2jO: 79
test/SXTvUsglT5/M4pR9zSHPA/Uab1nTe4X1/Na6gR2CtwxNpKQViUNQs.mp3 1LySIBy31q1mQ1b: 166
test/SXTvUsglT5/EVvSbvDMur/d5Quo9IlsI/TXGtTY4vtkKG8OEUJSWp.mp3 1fbNiCS6CNexkGp: 289
test/jc8mfAHTeq/c46xQSa2lF/02aHhvnF2d/jMwBUkeuo4Cq31mn4eUB.mp3 1gIbc1hRD665U5W: 548
test/CKVW6cIgxI/SKmnXqK9go/FLoSHSeHqP/HQWaCQnNvMv0t1BNmPoW.mp3 1p23gsb9WO94Bse: 181
test/K4EnkdRBeE/y1eVWk8N29/Uwb5RBHYZN/UiDJKYX8CqedbutySMWS.mp3 1vmk9ImBm9o3JfB

## 訓練データのノイズ除去

In [8]:
import time
import subprocess
import os
import pandas as pd

from pydub import AudioSegment
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

INPUT_PATH = "./data"

In [2]:
def remove_noise(input_file, output_file):
    # ディレクトリ部分を取得
    directory = os.path.dirname(output_file)

    # ディレクトリが存在しない場合は作成
    if not os.path.exists(directory):
        os.makedirs(directory)

    # FFmpegコマンドを作成
    command = [
        'ffmpeg',
        '-i', input_file,             # 入力ファイル
        '-af', 'afftdn=nf=-25', # "agate"    # ノイズ除去フィルタ (afftdn)
        output_file                   # 出力ファイル
    ]

    # FFmpegコマンドを実行
    subprocess.run(command, check=True)

In [3]:
!mkdir train_rm_noise
!mkdir train_vad_rm_noise
!ls

chutorial.ipynb			  poetry.lock	     train_llm_asr.py
data				  preprocess.ipynb   train_rm_noise
fine_tune			  pyproject.toml     train_vad_rm_noise
finetune_whisper.ipynb		  test		     train_whisper.py
inference_baseline_whisper.ipynb  test_rm_noise      vad.ipynb
inference_whisper.py		  test_vad	     vad_test.py
llm_asr				  test_vad_rm_noise  whisper_finetune
output				  train		     whisper_finetune_exp001


In [4]:
df_train = pd.read_csv(f"{INPUT_PATH}/train.csv")
df_train[0:3]

,ID,author,title,daisy_folder,audio_path,target
0,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...
1,06mgGNnTThgyIG6,w4mwiQxQI4,AYMK6FMxCG,9su1mGrvoq,train/w4mwiQxQI4/AYMK6FMxCG/9su1mGrvoq/pX3ipSW...,と自分は文政年間にできた地図で見たことがある小手指原久米川は古戦場なり源平小手指原にて戦うこ...
2,0MOLuKeb4EJvC45,PkyxkVV9Gq,cP82a8LULP,GZIf1u5UJu,train/PkyxkVV9Gq/cP82a8LULP/GZIf1u5UJu/ZXLCjee...,一あれ程迄世間を騒がせた天一坊も、とうとうお処刑となって、獄門に梟けられてしまいました。あの...


In [5]:
for audio_path in tqdm(df_train["audio_path"].to_list(), total=len(df_train["audio_path"].to_list())):
    # ノイズ除去
    rm_noise_audio_path = audio_path.replace("train", "train_rm_noise")
    remove_noise(audio_path, rm_noise_audio_path)

  0%|          | 0/800 [00:00<?, ?it/s]ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --

In [10]:
#訓練データの音声区切り目を付与
train_details = pd.read_csv(f"{INPUT_PATH}/train_details.csv")
df_train = pd.merge(df_train, train_details,on="ID")
df_train[0:2]

,ID,author,title,daisy_folder,audio_path,target,DETAIL_ID,slice_id,duration,start_time,end_time,target_slice
0,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...,022CmvRb8aM5pKF_1,1,2533.878,0.000,2533.878,幸ありて
1,022CmvRb8aM5pKF,4mLcmNwFwf,1TYGw0dUqi,guRpS8SYYB,train/4mLcmNwFwf/1TYGw0dUqi/guRpS8SYYB/oLMomBM...,幸ありて普通の人ならたいして問題にすまいこのことが、私の心を暗くした。もし耳がこのまま聞こえ...,022CmvRb8aM5pKF_2,2,3631.020,2533.878,6164.898,普通の人ならたいして問題にすまいこのことが、


In [11]:
# 音声を細切れ(slice)にして、別のディレクトリへ保存
def slice_audio(file_path, start_time, end_time, output_dir, output_filename):
    # Load the audio file
    audio = AudioSegment.from_mp3(file_path)

    # Slice the audio file (convert times to milliseconds)
    sliced_audio = audio[start_time:end_time]

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Construct the output file path
    output_path = os.path.join(output_dir, output_filename)

    # Export the sliced audio
    sliced_audio.export(output_path, format="mp3")

# 処理の実行
def process_audio_files(file_info):
    path, start_time, end_time, output_dir, output_filename = file_info
    slice_audio(path, start_time, end_time, output_dir, output_filename)

# 音声ファイルの情報を取得
list_audio_path = [i.replace("train", "train_rm_noise") for i in df_train["audio_path"].tolist()]
file_info_list = [
    (
        path,
        df_train.iloc[i, df_train.columns.get_loc("start_time")],
        df_train.iloc[i, df_train.columns.get_loc("end_time")],
        "train_vad_rm_noise/",
        df_train.iloc[i, df_train.columns.get_loc("DETAIL_ID")] + ".mp3"
    )
    for i, path in enumerate(list_audio_path)
]

start = time.time()

# 並列処理の設定（ProcessPoolExecutorを使用）
with ProcessPoolExecutor() as executor:
    list(tqdm(executor.map(process_audio_files, file_info_list), total=len(file_info_list)))

end = time.time()
print(f"処理時間: {end - start:.2f}秒")

100%|██████████| 354043/354043 [15:03:54<00:00,  6.53it/s]   


処理時間: 54250.32秒
